In [ ]:
import pandas as pd

data_path = '../../data/jeehoshin/foodcom_dataset/'
raw_recipe = pd.read_csv(data_path + 'RAW_recipes.csv')
ids = list(raw_recipe['id'])
print(len(ids))

raw_interaction = pd.read_csv(data_path + 'RAW_interactions.csv')
recipe_ids = list(set(list(raw_interaction['recipe_id'])))
print(len(recipe_ids))


import numpy as np
pd.set_option('display.max_columns', None)
np.random.seed(2024)
print("data preprocess start")
print("*"*30)

raw_interaction = pd.read_csv(data_path + 'RAW_interactions.csv')
pp_recipe = pd.read_csv(data_path + 'PP_recipes.csv')

records_removed = True
df = raw_interaction[raw_interaction['recipe_id'].isin(pp_recipe['id'].tolist())]

while records_removed:
    user_counts = df['user_id'].value_counts()
    item_counts = df['recipe_id'].value_counts()

    valid_users = user_counts[user_counts >= 5].index
    valid_items = item_counts[item_counts >= 5].index

    filtered_df = df[df['user_id'].isin(valid_users) & df['recipe_id'].isin(valid_items)]

    if len(filtered_df) == len(df):
        records_removed = False
    else:
        records_removed = True

    df = filtered_df
core_inter = df
sort_filter_ui = core_inter.sort_values('date').reset_index(drop=True)

231637
231637
data preprocess start
******************************


In [ ]:
import os
import re
import pandas as pd
import requests
from bs4 import BeautifulSoup
import socket
from tqdm import tqdm

Dir = data_path
image_dir = 'images/'

processed_recipe_ids = list(set(list(sort_filter_ui['recipe_id'])))
items = processed_recipe_ids  

raw_recipe = pd.read_csv(Dir + 'RAW_recipes.csv')
target_items = raw_recipe[raw_recipe['id'].isin(items)]

# Change to current dir
dst_dir = os.path.join(Dir, image_dir)
os.makedirs(dst_dir, exist_ok=True)
os.chdir(dst_dir)
print(f'Changed to Dir: {os.getcwd()}')
print('==============')

# URL formatting: Remove special characters
def format_image_url(recipe_id, name, url):
    safe_name = re.sub(r'[^a-zA-Z0-9\-]', '', name.replace(" ", "-").lower())
    return url + safe_name + '-' + str(recipe_id)

# Image download function
def dld_images(row):
    recipe_id = row['id']
    name = row['name']
    url = 'https://www.food.com/recipe/'
    url = format_image_url(recipe_id, name, url)

    header = {
        'User-Agent': (
            "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) "
            "AppleWebKit/537.36 (KHTML, like Gecko) "
            "Chrome/119.0.0.0 Safari/537.36"
        )
    }

    try:
        response = requests.get(url, headers=header, timeout=30)
        response.raise_for_status()
    except Exception as e:
        return f'{recipe_id}-page error: {e}'

    soup = BeautifulSoup(response.text, 'html.parser')

    # Find the first primary-image div
    current_recipe_divs = soup.find_all('div', class_='primary-image svelte-wgcq7z')
    if not current_recipe_divs:
        return f"{recipe_id}-no primary image div found"

    img_tag = current_recipe_divs[0].find('img')
    if img_tag is None or 'src' not in img_tag.attrs:
        return f"{recipe_id}-no img tag found"

    img_url = img_tag['src']
    ext = os.path.splitext(img_url)[-1].split('?')[0]
    if not ext:
        ext = ".jpg"
        
    img_filename = f"{recipe_id}{ext}" 
    if os.path.exists(img_filename):
        return f"{recipe_id}-image already exists"

    try:
        img_response = requests.get(img_url, headers=header, timeout=30)
        img_response.raise_for_status()
    except Exception as e:
        return f'{recipe_id}-image download error: {e}'

    with open(img_filename, 'wb') as f:
        f.write(img_response.content)

    return f"Downloaded {img_filename}"


############### MAIN ###############
target_items.reset_index(drop=True, inplace=True)

timeout = 45
socket.setdefaulttimeout(timeout)

results = []
for _, row in tqdm(target_items.iterrows(), total=len(target_items), desc="Downloading images"):
    result = dld_images(row)
    results.append(result)

print(f'\n============\n====== Finished =======')

# Check error messages
errors = [r for r in results if r and ("error" in r or "no" in r)]
if errors:
    print("\n⚠️ Errors:")
    for e in errors[:20]:  # Print only the first 20 if too many
        print(" -", e)


Changed to Dir: c:\Users\DMLab\Desktop\KAIRI_spring\recsys\foodcom\images



====== Finished =======

⚠️ Errors:
 - 106882-page error: 404 Client Error: Not Found for url: https://www.food.com/recipe/adapted-from-todd-wilbur-s-top-secret-recipes-big-mac-sauce-106882
 - 431946-page error: 404 Client Error: Not Found for url: https://www.food.com/recipe/america-s-test-kitchen-burgers-431946
 - 314560-page error: 404 Client Error: Not Found for url: https://www.food.com/recipe/america-s-test-kitchen-skillet-baked-ziti-314560
 - 292743-page error: 404 Client Error: Not Found for url: https://www.food.com/recipe/applebee-s-honey-pepper-sauce-for-salmon-copycat--by-todd-wilbur-292743
 - 4344-page error: 404 Client Error: Not Found for url: https://www.food.com/recipe/artichoke-spinach-dip-from-olive-garden-4344
 - 383039-image download error: 400 Client Error: Bad Request for url: https://img.sndimg.com/food/image/upload/f_auto,c_thumb,q_55,w_744,ar_5:4/v1/img/recipes/38/30/39/picXT6k4d.jpg
 - 5008-page error: 404 Client Error: Not Found for url: https://www.food.co

In [ ]:
print(os.getcwd())
print(f"Total files in {os.getcwd()}: {len(os.listdir())}")

file_counts = {}
for file in os.listdir():
    ext = os.path.splitext(file)[-1].lower()
    if ext not in file_counts:
        file_counts[ext] = 0
    file_counts[ext] += 1

for ext, count in file_counts.items():
    print(f"Files with {ext} extension: {count}")

# remove files that do not end with .jpg, .jpeg, .png
valid_extensions = {'.jpg', '.jpeg', '.png'}
all_files = os.listdir()
files_to_remove = [f for f in all_files if os.path.splitext(f)[-1].lower() not in valid_extensions]
for file in files_to_remove:
    os.remove(file)

print(f"Total files in {os.getcwd()}: {len(os.listdir())}")

c:\Users\DMLab\Desktop\KAIRI_spring\recsys\foodcom\images
Total files in c:\Users\DMLab\Desktop\KAIRI_spring\recsys\foodcom\images: 29943
Files with .jpg extension: 29103
Files with .jpeg extension: 764
Files with .png extension: 76
Total files in c:\Users\DMLab\Desktop\KAIRI_spring\recsys\foodcom\images: 29943
